In [1]:
import pandas as pd
import re
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import time
import os
from datetime import datetime

In [29]:
df = pd.read_csv('Waitrose.csv')

In [16]:
df.head()

,Name,Price,Price per Unit,Category,Subcategory,Date
0,Kissabel Apples Tray,2.55,Price per unit\n63.8p each,fresh_food,fruit,2024-11-19
1,Natures Candy USA Medjool Dates,5.00,Price per unit\n£11.02/kg,fresh_food,fruit,2024-11-19
2,Essential Fairtrade Bananas,0.20,Price per unit\n£1.10/kg,fresh_food,fruit,2024-11-19
3,Essential Fairtrade Bananas,0.95,Price per unit\n19p each,fresh_food,fruit,2024-11-19
4,Waitrose Seedless Easy Peelers,2.50,Price per unit\n£4.17/kg,fresh_food,fruit,2024-11-19


In [14]:
def clean_price(value):
    """
    Cleans the Price column by:
    - Handling pence (e.g., '90p / kg' or '90p') by removing 'p' and dividing by 100.
    - Handling pounds (e.g., '£2.50') by removing '£' and converting to float.
    """
    if isinstance(value, str):  # Ensure the value is a string
        value = value.strip()  # Remove any extra whitespace
        try:
            if 'p' in value:  # Case for pence
                return float(value.replace('p', '')) / 100
            elif '£' in value:  # Case for pounds
                return float(value.replace('£', ''))
        except ValueError:
            pass  # Ignore values that don't match expected formats
    return np.nan  # Return NaN for invalid or missing values

In [15]:
# Apply the cleaning function to the 'price' column
df['Price'] = df['Price'].apply(clean_price)

In [ ]:
def standardize_price_per_unit(price_per_unit):
    """
    Standardizes the 'Price per unit' column:
    - Removes the prefix 'Price per unit\n'.
    - Splits price and unit where applicable.
    - Converts prices to floats, handling 'p' and '£' cases.
    - Converts units like 'each' and recognizes 'other' units explicitly.
    """
    if isinstance(price_per_unit, str):  # Ensure the input is a string
        # Remove the prefix
        price_per_unit = price_per_unit.replace('Price per unit\n', '').strip()

        try:
            if '/' in price_per_unit:  # Split into price and unit
                price_value, unit = price_per_unit.split('/')
                price_value = price_value.replace(',', '').strip()  # Remove commas and whitespace

                # Handle price values with 'p' or '£'
                if 'p' in price_value:
                    price_value = float(price_value.replace('p', '')) / 100  # Pence to pounds
                elif '£' in price_value:
                    price_value = float(price_value.replace('£', ''))  # Pounds to float

                # Standardize units
                unit = unit.strip().lower()
                if 'each' in unit:
                    unit = 'each'
                elif '100g' in unit:
                    price_value *= 10  # 100g to kg 
                    unit = 'kg'
                elif '10g' in unit:
                    price_value *= 100  # 10g to kg
                    unit = 'kg'
                elif 'kg' in unit:
                    unit = 'kg'
                elif '100ml' in unit:
                    price_value *= 10  # 100ml to litre
                    unit = 'litre'
                elif 'litre' in unit:
                    unit = 'litre'
                elif 'cl' in unit:
                    price_value *= (4/3)  # Convert cl to litre
                    unit = 'litre'
                else:
                    unit = 'other'  # Unhandled units

                return price_value, unit

            elif 'each' in price_per_unit:  # Handle cases like '63.8p each' or '£5 each'
                price_value = price_per_unit.replace('each', '').strip()
                if 'p' in price_value:
                    price_value = float(price_value.replace('p', '')) / 100
                elif '£' in price_value:
                    price_value = float(price_value.replace('£', ''))
                return price_value, 'each'

            else:  # Handle unexpected formats
                return np.nan, 'other'
        except (ValueError, IndexError):
            return np.nan, 'other'  # Return defaults for malformed entries

    return np.nan, np.nan  # Return defaults for non-string inputs


In [30]:
# Apply the function to 'Price per Unit' column
df[['Standardised Price per Unit', 'Unit']] = df['Price per Unit'].apply(
    lambda x: pd.Series(standardize_price_per_unit(x))
)

# Verify the distinct units captured
print("Distinct Units:", df['Unit'].unique())

Distinct Units: ['each' 'kg' 'other' 'litre' nan]


In [31]:
# Filter rows where 'Unit' is 'other' or NaN
other_nan_rows = df[df['Unit'].isin(['other'])]


# Print the filtered rows
print("Rows with 'other':")
print(other_nan_rows)

Rows with 'other':
                                                   Name   Price  \
43                         Waitrose Large Vine Tomatoes   £4.00   
85                           No.1 Jack Hawkins Tomatoes   £5.55   
218                      Cooks' Ingredients Root Ginger   £6.20   
224                          Waitrose Echalion Shallots   £3.75   
229                                  Essential Brussels   £2.60   
280                               Waitrose Loose Fennel   £5.25   
284                                   Waitrose Shallots   £3.75   
303                         Waitrose Large Red Potatoes   £1.60   
306                             Waitrose Loose Celeriac   £1.60   
436                  Waitrose Flat Portabella Mushrooms   £5.00   
515                        Waitrose Loose Small Pumpkin   £2.20   
1440  No. 1 Cote Hill Lindum Semi Hard Cheese Streng...  £24.75   

              Price per Unit    Category Subcategory        Date  \
43    Price per unit\nper kg  fresh_food 